In [ ]:
import os
import sys
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Literal, Optional, Tuple, Union

import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import logging
from IPython.display import display
from matplotlib.axes import Axes as Axes

notebook_path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
project_dir = notebook_path.parent.parent
sys.path.append(str(project_dir))
import src.utils.custom_log as custom_log
from src.utils.Csv import Csv
from src.utils.iso18571 import rating_iso_18571
from src.utils.set_rcparams import set_rcparams
from src.utils.UnifySignal import UnifySignal

os.chdir(project_dir)
LOG = logging.getLogger(__name__)
custom_log.init_logger(log_lvl=logging.INFO)
LOG.info("Log initialized")

set_rcparams()

In [ ]:
B_PATH: Path = Path("/mnt") /"q" / "Val_Chain_Sims" / "AB_Testing"
B_PATH.is_dir()

In [ ]:
# read report data
CSV = Csv(csv_path=B_PATH / "From_Reports" / "report_data.csv.csv.zip", compress=True)
DB_REPORT: pd.DataFrame = CSV.read()
REPORT_ASS = ["HW TH Report", "CAE TH Report"]
DB_REPORT.replace({"CAE NHTSA": REPORT_ASS[1], "Test NHTSA": REPORT_ASS[0]}, inplace=True)
#DB_REPORT.fillna(0, inplace=True)
display(DB_REPORT)

In [ ]:
# read Other Data
DB_CAES: List[pd.DataFrame] = []
CASES: List[str] = [
    "000_Base_Model",
    "100_Guided_BIW",
    "200_PAB_Simplified",
    "300_Seat_Simplified",
    "400_HIII",
    "500_NoCAB",
    "600_NoDoor",
    "700_Simplified_Belt",
    "800_Simplified_BIW",
    "900_NoIntrusion",
    "950_Dash_Rigid",
    "990_Carpet_Rigid",
]
for case in CASES:
    LOG.info("Read %s", case)

    CSV = Csv(csv_path=B_PATH / case / f"extracted.csv.zip", compress=True)
    DB_CAES.append(CSV.read())


    DB_CAES[-1].replace(
        {"Assemblies_1": "SP 48", "Assemblies_2": "DP 48", "Assemblies_3": "SP 96", "Assemblies_4": "DP 96"}, inplace=True
    )

    DB_CAES[-1].fillna(0, inplace=True)

In [ ]:
DB_CAES[0]

In [ ]:
[x for x in DB_CAES[0]["Channel"].unique() if "03HEAD0000TH50ACR" in x]

In [ ]:
[x for x in DB_REPORT["Channel"].unique() if "03HEAD0000TH50ACR" in x]

In [ ]:
def plot():
    ch_name = "01HEAD0000TH50ACR"
    case = "Oblique Right"
    cs = {"HW TH Report": "black", "CAE TH Report": "blue", "CAE New": "orange"}
    names = {
        "HW TH Report": "Hardware-model Report (Singh et al. 2018a)",
        "CAE TH Report": "FE-model Report (Singh et al. 2018a)",
    }
    cfc_classes = {"D": 60, "B": 600, "C": 180}
    fig_width: float = 1 * (448.13095 / 72) -0.2
    fig_height: float = 0.4 * fig_width
    filters = ["B", "C", "D"]
    fig, ax = plt.subplot_mosaic(
        mosaic=[["L", "L", "L"], ["B", "C", "D"]],
        layout="constrained",
        height_ratios=[0.1, 1],
    )

    for cfc in filters:
        db = DB_CAES[0][
            DB_CAES[0]["Channel"].eq(f"{ch_name}{cfc}") & DB_CAES[0]["Case"].eq(case) & DB_CAES[0]["Assembly"].eq("SP 48")
        ]
        ax[cfc].plot(db["Time"], db["Value"], label="FE-model Initial with CFC 60-600", c=cs["CAE New"], alpha=1)

        for source in DB_REPORT["Source"].unique():
            db = DB_REPORT[DB_REPORT["Channel"].eq(f"{ch_name}D") & DB_REPORT["Source"].eq(source) & DB_REPORT["Case"].eq(case)]
            ax[cfc].plot(db["Time"], db["Value"], label=names[source], c=cs[source], alpha=0.7)

        ax[cfc].set_title(f"{cfc} - CFC {cfc_classes[cfc]}")
        ax[cfc].set_xlabel("Time [ms]")
        ax[cfc].grid()
        ax[cfc].set_xlim(0, 130)
        ax[cfc].set_xticks(np.arange(0, 121, 20))
        ax[cfc].set_ylim(0, 80)

    ax["L"].legend(*ax[filters[0]].get_legend_handles_labels(), loc="center", ncol=2)
    ax["L"].axis("off")
    ax[filters[0]].set_ylabel(ch_name, {"fontname": "CMU Typewriter Text", "fontsize": "large", "fontweight": "bold"})
    fig.set_figheight(fig_height)
    fig.set_figwidth(fig_width)
    fig.savefig(Path("reports") / "CFC_comp.pdf")


plot()